In [45]:
%pip install llama-index-llms-openai
!pip install langchain
!pip install llama_index

In [46]:
import nest_asyncio

nest_asyncio.apply()

In [47]:
from llama_index.core.evaluation import generate_question_context_pairs
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core.node_parser import SentenceSplitter
import llama_index.llms.openai
from llama_index.llms.openai import OpenAI

from langchain.document_loaders import TextLoader


In [1]:
# Loads scraped_NER.txt data file from /data folder
# https://drive.google.com/file/d/1kFwMeIhnmjirq-ejkOORDCJRKM6mg-7C/view?usp=drive_link
documents = SimpleDirectoryReader("./data/").load_data()

NameError: name 'SimpleDirectoryReader' is not defined

In [49]:
node_parser = SentenceSplitter(chunk_size=600)
nodes = node_parser.get_nodes_from_documents(documents)

In [50]:
# by default, the node ids are set to random uuids. To ensure same id's per run, we manually set them.
for idx, node in enumerate(nodes):
    node.id_ = f"node_{idx}"

In [52]:
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
import os
os.environ["OPENAI_API_KEY"] = "#####"
# openai.api_key = OPENAI_API_KEY

In [58]:
llm = OpenAI(model="gpt-3.5-turbo")

In [59]:
vector_index = VectorStoreIndex(nodes)

retriever = vector_index.as_retriever(similarity_top_k=2)

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [ ]:
from llama_index.core.evaluation import (
    generate_question_context_pairs,
    EmbeddingQAFinetuneDataset,
)

In [ ]:
qa_dataset = generate_question_context_pairs(
    nodes, llm=llm, num_questions_per_chunk=2
)

In [ ]:
queries = qa_dataset.queries.values()
print(list(queries)[2])

In [ ]:
# [optional] save
qa_dataset.save_json("pg_eval_dataset.json")

In [ ]:
# [optional] load
qa_dataset = EmbeddingQAFinetuneDataset.from_json("pg_eval_dataset.json")

In [ ]:
include_cohere_rerank = True

if include_cohere_rerank:
    !pip install cohere -q

In [ ]:
from llama_index.core.evaluation import RetrieverEvaluator

metrics = ["mrr", "hit_rate"]

if include_cohere_rerank:
    metrics.append(
        "cohere_rerank_relevancy"  # requires COHERE_API_KEY environment variable to be set
    )

retriever_evaluator = RetrieverEvaluator.from_metric_names(
    metrics, retriever=retriever
)

In [ ]:
# try it out on a sample query
sample_id, sample_query = list(qa_dataset.queries.items())[0]
sample_expected = qa_dataset.relevant_docs[sample_id]

eval_result = retriever_evaluator.evaluate(sample_query, sample_expected)
print(eval_result)

In [ ]:
# try it out on an entire dataset
eval_results = await retriever_evaluator.aevaluate_dataset(qa_dataset)

In [ ]:
import pandas as pd


def display_results(name, eval_results):
    """Display results from evaluate."""

    metric_dicts = []
    for eval_result in eval_results:
        metric_dict = eval_result.metric_vals_dict
        metric_dicts.append(metric_dict)

    full_df = pd.DataFrame(metric_dicts)

    hit_rate = full_df["hit_rate"].mean()
    mrr = full_df["mrr"].mean()
    columns = {"retrievers": [name], "hit_rate": [hit_rate], "mrr": [mrr]}

    if include_cohere_rerank:
        crr_relevancy = full_df["cohere_rerank_relevancy"].mean()
        columns.update({"cohere_rerank_relevancy": [crr_relevancy]})

    metric_df = pd.DataFrame(columns)

    return metric_df